In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import numba
from numba import jit

In [14]:
infname = "C:/repos/globiom_clim_extract/input_cmip/CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn.nc"

In [32]:
data = xr.open_dataarray(infname)
arr = data.data
arr.shape

(1, 480, 160, 320)

In [28]:
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [33]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(arr): # Function is compiled to machine code when called the first time
    out = arr
    for i in range(arr.shape[0]):  
        for j in range(arr.shape[1]):  
            for k in range(arr.shape[2]):  
                for l in range(arr.shape[3]):  
                    out[i,j,k,l] = arr[i,j,k,l]*2
    return out              # Numba likes NumPy broadcasting

In [24]:
range(arr.shape[1]-1)

range(0, 479)

In [35]:
print(go_fast(arr))

[[[[ 976.2688   976.188    976.07983 ...  976.5454   976.44336
     976.3672 ]
   [ 979.23236  978.98553  978.7408  ...  979.97107  979.72327
     979.4849 ]
   [ 981.685    981.3039   980.9275  ...  982.9021   982.4675
     982.08075]
   ...
   [ 964.24084  964.24756  964.2643  ...  964.27765  964.26465
     964.24164]
   [ 965.7704   965.7771   965.77545 ...  965.68536  965.7172
     965.7453 ]
   [ 967.37024  967.39795  967.42706 ...  967.3574   967.3608
     967.36414]]

  [[ 948.88556  948.95874  948.5761  ...  949.3883   949.12317
     949.01355]
   [ 950.18054  949.8776   949.58466 ...  951.15436  950.80286
     950.505  ]
   [ 950.5368   950.0443   949.5131  ...  952.17285  951.59515
     951.0521 ]
   ...
   [ 964.7167   964.7731   964.8417  ...  964.57465  964.60205
     964.6521 ]
   [ 967.4611   967.5094   967.57605 ...  967.2707   967.3454
     967.4073 ]
   [ 969.0947   969.1613   969.23065 ...  969.0807   969.08295
     969.0852 ]]

  [[ 893.9405   893.84515  893.77515 .